Import Libraries


In [1]:
import numpy as np
from osgeo import gdal
from scipy import stats

<img src="data/thumb.png" alt="Drawing" style="width: 300px;"/>

Load test data

In [2]:
r = gdal.Open("data\\nlcd_2001_landcover_2011_edition_2014_10_sub.tif")
x = np.array(r.GetRasterBand(1).ReadAsArray())



In [9]:
#stats.describe(x) 
x.shape
out = np.zeros(x.shape)

Function to create data structure function takes in a grid with mxn and returns a grid with mxnxc where c is the number of classes.


In [17]:
def dtFZS(g):
    out = np.zeros(g.shape,dtype='uint64')
    tmp = 0
    for xw in g.shape[0]
        for yw in g.shape[1]
            out = out[]

In [18]:
dtFZS(x)